In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm

## 0. Import libraries and read in data

In [2]:
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
from tqdm import tqdm

In [3]:
df = pd.read_csv('../data/raw/sales_train_validation.csv')
price_df = pd.read_csv('../data/raw/sell_prices.csv')
cal_df = pd.read_csv('../data/raw/calendar.csv')

In [4]:
cal_df["d"]=cal_df["d"].apply(lambda x: int(x.split("_")[1]))
price_df["id"] = price_df["item_id"] + "_" + price_df["store_id"] + "_validation"

In [5]:
df.head()

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,3,0,1,1,1,3,0,1,1
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,1,2,1,1,1,0,1,1,1
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,0,5,4,1,0,1,3,7,2
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,1,1,0,1,1,2,2,2,4


## 1. Calculate weight for the level 12 series

In [6]:
for day in tqdm(range(1886, 1914)):
    wk_id = list(cal_df[cal_df["d"]==day]["wm_yr_wk"])[0]
    wk_price_df = price_df[price_df["wm_yr_wk"]==wk_id]
    df = df.merge(wk_price_df[["sell_price", "id"]], on=["id"], how='inner')
    df["unit_sales_" + str(day)] = df["sell_price"] * df["d_" + str(day)]
    df.drop(columns=["sell_price"], inplace=True)

100%|██████████████████████████████████████████████████████████████████████████████████| 28/28 [00:37<00:00,  1.35s/it]


In [7]:
df["dollar_sales"] = df[[c for c in df.columns if c.find("unit_sales")==0]].sum(axis=1)

In [8]:
df.drop(columns=[c for c in df.columns if c.find("unit_sales")==0], inplace=True)

In [9]:
df["weight"] = df["dollar_sales"] / df["dollar_sales"].sum()

In [10]:
df['dollar_sales'].head()

0    224.94
1      7.94
2     47.52
3    236.64
4    109.44
Name: dollar_sales, dtype: float64

In [11]:
df.head()

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913,dollar_sales,weight
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,1,1,1,3,0,1,1,224.94,0.000060
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,1,0,0,0,0,7.94,0.000002
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,1,1,1,0,1,1,1,47.52,0.000013
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,5,4,1,0,1,3,7,2,236.64,0.000063
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,0,1,1,2,2,2,4,109.44,0.000029


In [12]:
df.drop(columns=["dollar_sales"], inplace=True)

In [13]:
df["weight"] /= 12

In [14]:
df.head()

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913,weight
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,3,0,1,1,1,3,0,1,1,5.001094e-06
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,0,1,0,0,0,0,1.765301e-07
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,2,1,1,1,0,1,1,1,1.056513e-06
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,5,4,1,0,1,3,7,2,5.261220e-06
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,1,0,1,1,2,2,2,4,2.433181e-06


## 2. Transform time series

In [15]:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))

In [16]:
df.dtypes

id           object
item_id      object
dept_id      object
cat_id       object
store_id     object
             ...   
d_1910        int64
d_1911        int64
d_1912        int64
d_1913        int64
weight      float64
Length: 1920, dtype: object

In [17]:
print('Largest sales value is', df[[c for c in df.columns if c.find('d_')==0]].max().max(), 
      '\nLargest int16 is', np.iinfo(np.int16).max)

Largest sales value is 763 
Largest int16 is 32767


In [18]:
for d in range(1914, 1942):
    df["d_" + str(d)] = np.nan

In [19]:
df.head()

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1932,d_1933,d_1934,d_1935,d_1936,d_1937,d_1938,d_1939,d_1940,d_1941
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
df.drop(columns = [c for c in df.columns if c.find('d_')==0 and int(c.split('_')[1]) < 1100])

,id,item_id,dept_id,cat_id,store_id,state_id,d_1100,d_1101,d_1102,d_1103,...,d_1932,d_1933,d_1934,d_1935,d_1936,d_1937,d_1938,d_1939,d_1940,d_1941
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,1,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,1,0,0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,4,2,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,3,1,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30485,FOODS_3_823_WI_3_validation,FOODS_3_823,FOODS_3,FOODS,WI_3,WI,0,2,1,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
30486,FOODS_3_824_WI_3_validation,FOODS_3_824,FOODS_3,FOODS,WI_3,WI,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
30487,FOODS_3_825_WI_3_validation,FOODS_3_825,FOODS_3,FOODS,WI_3,WI,0,2,2,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
30488,FOODS_3_826_WI_3_validation,FOODS_3_826,FOODS_3,FOODS,WI_3,WI,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
temp = df.drop(columns = [c for c in df.columns if c.find('d_')==0 and int(c.split('_')[1]) < 1100])

In [22]:
temp.melt(id_vars=[n for n in temp.columns if n.find("id")!=-1],
       value_vars=[n for n in temp.columns if n.find("d_")==0],
       var_name = 'day', value_name = 'sales')

,id,item_id,dept_id,cat_id,store_id,state_id,day,sales
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_1100,1.0
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,d_1100,1.0
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,d_1100,0.0
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,d_1100,4.0
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,d_1100,3.0
...,...,...,...,...,...,...,...,...
25672575,FOODS_3_823_WI_3_validation,FOODS_3_823,FOODS_3,FOODS,WI_3,WI,d_1941,NaN
25672576,FOODS_3_824_WI_3_validation,FOODS_3_824,FOODS_3,FOODS,WI_3,WI,d_1941,NaN
25672577,FOODS_3_825_WI_3_validation,FOODS_3_825,FOODS_3,FOODS,WI_3,WI,d_1941,NaN
25672578,FOODS_3_826_WI_3_validation,FOODS_3_826,FOODS_3,FOODS,WI_3,WI,d_1941,NaN


In [23]:
df_melted = temp.melt(id_vars=[n for n in temp.columns if n.find("id")!=-1],
       value_vars=[n for n in temp.columns if n.find("d_")==0],
       var_name = 'day', value_name = 'sales')
del temp

In [24]:
df[['id', 'd_1100']].head(1)

,id,d_1100
0,HOBBIES_1_001_CA_1_validation,1


In [25]:
df_melted.head()

,id,item_id,dept_id,cat_id,store_id,state_id,day,sales
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_1100,1.0
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,d_1100,1.0
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,d_1100,0.0
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,d_1100,4.0
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,d_1100,3.0


In [26]:
df_melted.tail()

,id,item_id,dept_id,cat_id,store_id,state_id,day,sales
25672575,FOODS_3_823_WI_3_validation,FOODS_3_823,FOODS_3,FOODS,WI_3,WI,d_1941,NaN
25672576,FOODS_3_824_WI_3_validation,FOODS_3_824,FOODS_3,FOODS,WI_3,WI,d_1941,NaN
25672577,FOODS_3_825_WI_3_validation,FOODS_3_825,FOODS_3,FOODS,WI_3,WI,d_1941,NaN
25672578,FOODS_3_826_WI_3_validation,FOODS_3_826,FOODS_3,FOODS,WI_3,WI,d_1941,NaN
25672579,FOODS_3_827_WI_3_validation,FOODS_3_827,FOODS_3,FOODS,WI_3,WI,d_1941,NaN


In [27]:
df_melted["day"]=df_melted["day"].apply(lambda x: int(x.split("_")[1]))

In [28]:
df_melted.head()

,id,item_id,dept_id,cat_id,store_id,state_id,day,sales
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,1100,1.0
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,1100,1.0
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,1100,0.0
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,1100,4.0
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,1100,3.0


In [29]:
df_melted=df_melted.merge(cal_df.drop(columns=["date", "wm_yr_wk", 
                                            "weekday"]), left_on=["day"], right_on=["d"]).drop(columns=["d"])

In [30]:
df_melted.head()

,id,item_id,dept_id,cat_id,store_id,state_id,day,sales,wday,month,year,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,1100,1.0,1,2,2014,NaN,NaN,NaN,NaN,1,1,0
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,1100,1.0,1,2,2014,NaN,NaN,NaN,NaN,1,1,0
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,1100,0.0,1,2,2014,NaN,NaN,NaN,NaN,1,1,0
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,1100,4.0,1,2,2014,NaN,NaN,NaN,NaN,1,1,0
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,1100,3.0,1,2,2014,NaN,NaN,NaN,NaN,1,1,0


In [31]:
df_melted['event_name_1'].value_counts(dropna=False).head()

NaN              23538280
Cinco De Mayo       91470
Mother's day        91470
Pesach End          91470
LentWeek2           91470
Name: event_name_1, dtype: int64

In [32]:
df_melted.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25672580 entries, 0 to 25672579
Data columns (total 18 columns):
 #   Column        Dtype  
---  ------        -----  
 0   id            object 
 1   item_id       object 
 2   dept_id       object 
 3   cat_id        object 
 4   store_id      object 
 5   state_id      object 
 6   day           int64  
 7   sales         float64
 8   wday          int64  
 9   month         int64  
 10  year          int64  
 11  event_name_1  object 
 12  event_type_1  object 
 13  event_name_2  object 
 14  event_type_2  object 
 15  snap_CA       int64  
 16  snap_TX       int64  
 17  snap_WI       int64  
dtypes: float64(1), int64(7), object(10)
memory usage: 3.6+ GB


In [33]:
# casting the categorical data of event into int8 and then taking count
df_melted['event_name_1'].astype('category').cat.codes.astype("int8").value_counts().head()

-1     23538280
 22       91470
 2        91470
 4        91470
 11       91470
dtype: int64

In [34]:
df_melted["event_name_1"]=df_melted["event_name_1"].astype('category').cat.codes.astype("int8")
df_melted["event_name_2"]=df_melted["event_name_2"].astype('category').cat.codes.astype("int8")
df_melted["event_type_1"]=df_melted["event_type_1"].astype('category').cat.codes.astype("int8")
df_melted["event_type_2"]=df_melted["event_type_2"].astype('category').cat.codes.astype("int8")

In [35]:
df_melted.head()

,id,item_id,dept_id,cat_id,store_id,state_id,day,sales,wday,month,year,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,1100,1.0,1,2,2014,-1,-1,-1,-1,1,1,0
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,1100,1.0,1,2,2014,-1,-1,-1,-1,1,1,0
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,1100,0.0,1,2,2014,-1,-1,-1,-1,1,1,0
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,1100,4.0,1,2,2014,-1,-1,-1,-1,1,1,0
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,1100,3.0,1,2,2014,-1,-1,-1,-1,1,1,0


In [36]:
df_melted.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25672580 entries, 0 to 25672579
Data columns (total 18 columns):
 #   Column        Dtype  
---  ------        -----  
 0   id            object 
 1   item_id       object 
 2   dept_id       object 
 3   cat_id        object 
 4   store_id      object 
 5   state_id      object 
 6   day           int64  
 7   sales         float64
 8   wday          int64  
 9   month         int64  
 10  year          int64  
 11  event_name_1  int8   
 12  event_type_1  int8   
 13  event_name_2  int8   
 14  event_type_2  int8   
 15  snap_CA       int64  
 16  snap_TX       int64  
 17  snap_WI       int64  
dtypes: float64(1), int64(7), int8(4), object(6)
memory usage: 3.0+ GB


In [37]:
# converting other categorical columns to int8 and int16
useful_ids = ['item_id', 'dept_id', 'cat_id', 'store_id', 'state_id']
id_encodings = [id_col + '_encoding' for id_col in useful_ids]

for id_col in useful_ids:
    if id_col == 'item_id':
        df_melted[id_col + '_encoding'] = df_melted[id_col].astype('category').cat.codes.astype("int16")
    else: 
        df_melted[id_col + '_encoding'] = df_melted[id_col].astype('category').cat.codes.astype("int8")

In [38]:
df_melted.head()

,id,item_id,dept_id,cat_id,store_id,state_id,day,sales,wday,month,...,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,item_id_encoding,dept_id_encoding,cat_id_encoding,store_id_encoding,state_id_encoding
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,1100,1.0,1,2,...,-1,-1,1,1,0,1437,3,1,0,0
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,1100,1.0,1,2,...,-1,-1,1,1,0,1438,3,1,0,0
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,1100,0.0,1,2,...,-1,-1,1,1,0,1439,3,1,0,0
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,1100,4.0,1,2,...,-1,-1,1,1,0,1440,3,1,0,0
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,1100,3.0,1,2,...,-1,-1,1,1,0,1441,3,1,0,0


In [39]:
df_melted.drop(columns=['year'] + useful_ids, inplace=True)

In [40]:
df_melted.head()

,id,day,sales,wday,month,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,item_id_encoding,dept_id_encoding,cat_id_encoding,store_id_encoding,state_id_encoding
0,HOBBIES_1_001_CA_1_validation,1100,1.0,1,2,-1,-1,-1,-1,1,1,0,1437,3,1,0,0
1,HOBBIES_1_002_CA_1_validation,1100,1.0,1,2,-1,-1,-1,-1,1,1,0,1438,3,1,0,0
2,HOBBIES_1_003_CA_1_validation,1100,0.0,1,2,-1,-1,-1,-1,1,1,0,1439,3,1,0,0
3,HOBBIES_1_004_CA_1_validation,1100,4.0,1,2,-1,-1,-1,-1,1,1,0,1440,3,1,0,0
4,HOBBIES_1_005_CA_1_validation,1100,3.0,1,2,-1,-1,-1,-1,1,1,0,1441,3,1,0,0


In [41]:
reduce_mem_usage(df_melted)

Mem. usage decreased to 856.91 Mb (57.3% reduction)


In [42]:
df_melted[["id","sales"]].groupby("id")["sales"].head()

0         1.0
1         1.0
2         0.0
3         4.0
4         3.0
         ... 
152445    0.0
152446    0.0
152447    0.0
152448    0.0
152449    0.0
Name: sales, Length: 152450, dtype: float16

In [43]:
df_melted[["id","sales"]].groupby("id")["sales"].shift(1).head()

0   NaN
1   NaN
2   NaN
3   NaN
4   NaN
Name: sales, dtype: float16

In [44]:
df_melted[["id","sales"]].groupby("id")["sales"].shift(1).fillna(-1).head()

0   -1.0
1   -1.0
2   -1.0
3   -1.0
4   -1.0
Name: sales, dtype: float16

In [45]:
df_melted[["id","sales"]].groupby("id")["sales"].shift(1).fillna(-1).astype(np.int16).head()

0   -1
1   -1
2   -1
3   -1
4   -1
Name: sales, dtype: int16

In [46]:
df_melted['test'] = df_melted[["id","sales"]].groupby("id")["sales"].shift(1).fillna(-1).astype(np.int16)

In [47]:
print(list(df_melted[df_melted['day']==1101]['test']) == list(df_melted[df_melted['day']==1100]['sales']))

True


In [48]:
df_melted.head()

,id,day,sales,wday,month,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,item_id_encoding,dept_id_encoding,cat_id_encoding,store_id_encoding,state_id_encoding,test
0,HOBBIES_1_001_CA_1_validation,1100,1.0,1,2,-1,-1,-1,-1,1,1,0,1437,3,1,0,0,-1
1,HOBBIES_1_002_CA_1_validation,1100,1.0,1,2,-1,-1,-1,-1,1,1,0,1438,3,1,0,0,-1
2,HOBBIES_1_003_CA_1_validation,1100,0.0,1,2,-1,-1,-1,-1,1,1,0,1439,3,1,0,0,-1
3,HOBBIES_1_004_CA_1_validation,1100,4.0,1,2,-1,-1,-1,-1,1,1,0,1440,3,1,0,0,-1
4,HOBBIES_1_005_CA_1_validation,1100,3.0,1,2,-1,-1,-1,-1,1,1,0,1441,3,1,0,0,-1


In [49]:
df_melted.drop(columns=['test'], inplace=True)

In [50]:
# create lags, lags starts from 28 days ago to 77 days ago, spaced by 7 days
for lag in tqdm([28, 35, 42, 49, 56, 63, 70, 77]):
    df_melted["lag_" + str(lag)] = df_melted[["id","sales"]].groupby("id")["sales"].shift(lag).fillna(-1).astype(np.int16)

100%|████████████████████████████████████████████████████████████████████████████████████| 8/8 [01:41<00:00, 12.72s/it]


In [52]:
df_melted.head()

,id,day,sales,wday,month,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,...,store_id_encoding,state_id_encoding,lag_28,lag_35,lag_42,lag_49,lag_56,lag_63,lag_70,lag_77
0,HOBBIES_1_001_CA_1_validation,1100,1.0,1,2,-1,-1,-1,-1,1,...,0,0,-1,-1,-1,-1,-1,-1,-1,-1
1,HOBBIES_1_002_CA_1_validation,1100,1.0,1,2,-1,-1,-1,-1,1,...,0,0,-1,-1,-1,-1,-1,-1,-1,-1
2,HOBBIES_1_003_CA_1_validation,1100,0.0,1,2,-1,-1,-1,-1,1,...,0,0,-1,-1,-1,-1,-1,-1,-1,-1
3,HOBBIES_1_004_CA_1_validation,1100,4.0,1,2,-1,-1,-1,-1,1,...,0,0,-1,-1,-1,-1,-1,-1,-1,-1
4,HOBBIES_1_005_CA_1_validation,1100,3.0,1,2,-1,-1,-1,-1,1,...,0,0,-1,-1,-1,-1,-1,-1,-1,-1


In [51]:
# all those rows where lag_77 is not -1
df_melted[df_melted['lag_77']!=-1].head()

,id,day,sales,wday,month,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,...,store_id_encoding,state_id_encoding,lag_28,lag_35,lag_42,lag_49,lag_56,lag_63,lag_70,lag_77
2347730,HOBBIES_1_001_CA_1_validation,1177,2.0,1,4,-1,-1,-1,-1,0,...,0,0,1,0,0,1,2,1,2,1
2347731,HOBBIES_1_002_CA_1_validation,1177,1.0,1,4,-1,-1,-1,-1,0,...,0,0,0,0,1,1,0,0,0,1
2347732,HOBBIES_1_003_CA_1_validation,1177,1.0,1,4,-1,-1,-1,-1,0,...,0,0,0,0,0,2,0,1,0,0
2347733,HOBBIES_1_004_CA_1_validation,1177,3.0,1,4,-1,-1,-1,-1,0,...,0,0,5,2,6,0,12,5,4,4
2347734,HOBBIES_1_005_CA_1_validation,1177,2.0,1,4,-1,-1,-1,-1,0,...,0,0,1,2,2,0,1,1,0,3


In [53]:
df_melted=df_melted[df_melted['lag_77']!=-1]

In [54]:
assert list(df_melted[df_melted['day']==1528]['lag_28']) == list(df_melted[df_melted['day']==1500]['sales'])

In [55]:
df_melted.head()

,id,day,sales,wday,month,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,...,store_id_encoding,state_id_encoding,lag_28,lag_35,lag_42,lag_49,lag_56,lag_63,lag_70,lag_77
2347730,HOBBIES_1_001_CA_1_validation,1177,2.0,1,4,-1,-1,-1,-1,0,...,0,0,1,0,0,1,2,1,2,1
2347731,HOBBIES_1_002_CA_1_validation,1177,1.0,1,4,-1,-1,-1,-1,0,...,0,0,0,0,1,1,0,0,0,1
2347732,HOBBIES_1_003_CA_1_validation,1177,1.0,1,4,-1,-1,-1,-1,0,...,0,0,0,0,0,2,0,1,0,0
2347733,HOBBIES_1_004_CA_1_validation,1177,3.0,1,4,-1,-1,-1,-1,0,...,0,0,5,2,6,0,12,5,4,4
2347734,HOBBIES_1_005_CA_1_validation,1177,2.0,1,4,-1,-1,-1,-1,0,...,0,0,1,2,2,0,1,1,0,3


In [56]:
price_df.head()

,store_id,item_id,wm_yr_wk,sell_price,id
0,CA_1,HOBBIES_1_001,11325,9.58,HOBBIES_1_001_CA_1_validation
1,CA_1,HOBBIES_1_001,11326,9.58,HOBBIES_1_001_CA_1_validation
2,CA_1,HOBBIES_1_001,11327,8.26,HOBBIES_1_001_CA_1_validation
3,CA_1,HOBBIES_1_001,11328,8.26,HOBBIES_1_001_CA_1_validation
4,CA_1,HOBBIES_1_001,11329,8.26,HOBBIES_1_001_CA_1_validation


In [57]:
df_melted=df_melted.merge(cal_df[['d', 'wm_yr_wk']], left_on=['day'], right_on=['d']).drop(columns=['d'])

In [58]:
df_melted.head()

,id,day,sales,wday,month,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,...,state_id_encoding,lag_28,lag_35,lag_42,lag_49,lag_56,lag_63,lag_70,lag_77,wm_yr_wk
0,HOBBIES_1_001_CA_1_validation,1177,2.0,1,4,-1,-1,-1,-1,0,...,0,1,0,0,1,2,1,2,1,11412
1,HOBBIES_1_002_CA_1_validation,1177,1.0,1,4,-1,-1,-1,-1,0,...,0,0,0,1,1,0,0,0,1,11412
2,HOBBIES_1_003_CA_1_validation,1177,1.0,1,4,-1,-1,-1,-1,0,...,0,0,0,0,2,0,1,0,0,11412
3,HOBBIES_1_004_CA_1_validation,1177,3.0,1,4,-1,-1,-1,-1,0,...,0,5,2,6,0,12,5,4,4,11412
4,HOBBIES_1_005_CA_1_validation,1177,2.0,1,4,-1,-1,-1,-1,0,...,0,1,2,2,0,1,1,0,3,11412


In [59]:
df_melted=df_melted.merge(price_df[['id', 'sell_price', 'wm_yr_wk']], on=['id', 'wm_yr_wk'], how='inner')

In [60]:
df_melted.head()

,id,day,sales,wday,month,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,...,lag_28,lag_35,lag_42,lag_49,lag_56,lag_63,lag_70,lag_77,wm_yr_wk,sell_price
0,HOBBIES_1_001_CA_1_validation,1177,2.0,1,4,-1,-1,-1,-1,0,...,1,0,0,1,2,1,2,1,11412,8.26
1,HOBBIES_1_001_CA_1_validation,1178,0.0,2,4,4,0,1,1,0,...,0,1,1,0,0,2,0,0,11412,8.26
2,HOBBIES_1_001_CA_1_validation,1179,1.0,3,4,-1,-1,-1,-1,0,...,2,0,0,0,0,0,0,0,11412,8.26
3,HOBBIES_1_001_CA_1_validation,1180,0.0,4,4,21,2,-1,-1,0,...,0,1,0,1,1,0,0,0,11412,8.26
4,HOBBIES_1_001_CA_1_validation,1181,0.0,5,4,-1,-1,-1,-1,0,...,1,0,0,0,0,0,0,0,11412,8.26


In [62]:
df_melted.info()

MemoryError: Unable to allocate 282. MiB for an array with shape (13, 22783757) and data type int8

In [64]:
df_melted.to_pickle('../data/processed/df_melted.pkl')

MemoryError: 

In [ ]:
del price_df
del cal_df

In [1]:
import lightgbm as lgb

In [ ]:
best_params = {
            "objective" : "poisson",
            "metric" :"rmse",
            "force_row_wise" : True,
            "learning_rate" : 0.05,
    #         "sub_feature" : 0.8,
            "sub_row" : 0.75,
            "bagging_freq" : 1,
            "lambda_l2" : 0.1,
    #         "nthread" : 4
            "metric": ["rmse"],
        'verbosity': 1,
        'num_iterations' : 2048,
        'num_leaves': 64,
        "min_data_in_leaf": 50,
    }

In [ ]:
del wk_price_df

In [ ]:
X_train = df_melted[df_melted["day"] < 1886].drop(columns=["sales"])
X_val = df_melted[df_melted["day"].between(1886, 1913)].drop(columns=["sales"])
X_test = df_melted[df_melted["day"] > 1913].drop(columns=["sales"])

y_train = df_melted[df_melted["day"] < 1886]["sales"]
y_val = df_melted[df_melted["day"].between(1886, 1913)]["sales"]

In [ ]:
%who DataFrame

In [ ]:
del df_melted

In [ ]:
X_train.columns

In [ ]:
%%time

np.random.seed(777)

fake_valid_inds = np.random.choice(X_train.index.values, 2_000_000, replace = False)
train_inds = np.setdiff1d(X_train.index.values, fake_valid_inds)
train_data = lgb.Dataset(X_train.drop(columns=['id']).loc[train_inds] , label = y_train.loc[train_inds], 
                         categorical_feature=id_encodings, free_raw_data=False)
fake_valid_data = lgb.Dataset(X_train.drop(columns=['id']).loc[fake_valid_inds], label = y_train.loc[fake_valid_inds],
                              categorical_feature=id_encodings,
                 free_raw_data=False)# This is a random sample, we're not gonna apply any time series train-test-split tricks here!

In [ ]:
%%time

m_lgb = lgb.train(best_params, train_data, valid_sets = [fake_valid_data], verbose_eval=1) 

In [ ]:
for d in range(1886, 1914):
    df['F_' + str(d)] = m_lgb.predict(X_val[X_val['day']==d].drop(columns=['id']))

In [ ]:
df.head()

In [ ]:
h = 28
n = 1885
def rmsse(ground_truth, forecast, train_series, axis=1):
    # assuming input are numpy array or matrices
    assert axis == 0 or axis == 1
    assert type(ground_truth) == np.ndarray and type(forecast) == np.ndarray and type(train_series) == np.ndarray
    
    if axis == 1:
        # using axis == 1 we must guarantee these are matrices and not arrays
        assert ground_truth.shape[1] > 1 and forecast.shape[1] > 1 and train_series.shape[1] > 1
    
    numerator = ((ground_truth - forecast)**2).sum(axis=axis)
    if axis == 1:
        denominator = 1/(n-1) * ((train_series[:, 1:] - train_series[:, :-1]) ** 2).sum(axis=axis)
    else:
        denominator = 1/(n-1) * ((train_series[1:] - train_series[:-1]) ** 2).sum(axis=axis)
    if (numerator < 0).any():
        print('nu')
    elif (denominator < 0).any():
        print(denominator[denominator < 0])
    return (1/h * numerator/denominator) ** 0.5

In [ ]:
level_groupings = {2: ["state_id"], 3: ["store_id"], 4: ["cat_id"], 5: ["dept_id"], 
              6: ["state_id", "cat_id"], 7: ["state_id", "dept_id"], 8: ["store_id", "cat_id"], 9: ["store_id", "dept_id"],
              10: ["item_id"], 11: ["item_id", "state_id"]}

In [ ]:
#remake agg_df
new_agg_df = pd.DataFrame(df[[c for c in df.columns if c.find("d_") == 0 or c.find("F_") == 0]].sum()).transpose()
new_agg_df["level"] = 1
new_agg_df["weight"] = 1/12
column_order = new_agg_df.columns

for level in level_groupings:
    temp_df = df.groupby(by=level_groupings[level]).sum().reset_index()
    temp_df["level"] = level
    new_agg_df = new_agg_df.append(temp_df[column_order])
del temp_df

agg_df = new_agg_df

train_series_cols = [c for c in df.columns if c.find("d_") == 0 and int(c.split('_')[1]) < 1886]
ground_truth_cols = [c for c in df.columns if c.find("d_") == 0 and int(c.split('_')[1]) in range(1886, 1914)]
forecast_cols = [c for c in df.columns if c.find("F_") == 0]

df["rmsse"] = rmsse(np.array(df[ground_truth_cols]), 
        np.array(df[forecast_cols]), np.array(df[train_series_cols]))
agg_df["rmsse"] = rmsse(np.array(agg_df[ground_truth_cols]), 
        np.array(agg_df[forecast_cols]), np.array(agg_df[train_series_cols]))

df["wrmsse"] = df["weight"] * df["rmsse"]
agg_df["wrmsse"] = agg_df["weight"] * agg_df["rmsse"]

print(df["wrmsse"].sum() + agg_df["wrmsse"].sum())

In [ ]:
lgb.plot_importance(m_lgb)

###### Make submission file

In [ ]:
submit_df = df[["id"]]
for i in range(1, 29):
    submit_df["F" + str(i)] = m_lgb.predict(X_test[X_test['day']==i+1913].drop(columns=['id']))

In [ ]:
submit_df.head()

In [ ]:
submit_df2 = submit_df.copy()
submit_df2["id"] = submit_df2["id"].apply(lambda x: x.replace('validation',
                                                              'evaluation'))

In [ ]:
submit_df = submit_df.append(submit_df2).reset_index(drop=True)

In [ ]:
submit_df.to_csv("submission.csv", index=False)